In [1]:
from zntrack import Node, dvc, config, zn

In [2]:
config.nb_name = "06_named_nodes.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmpbhmmn615/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


# Named Nodes
Named Nodes allow us to use the same Node multiple times in a single graph at e.g. different steps. Therefore, we can pass a `name` argument to the `__init__` of our Node.

<blockquote>Notice that this is one of only very few scenarios where we want to pass an argument directly to the `__init__`</blockquote>

In [5]:
class HelloWorld(Node):
    inputs = zn.params()
    outputs = zn.outs()

    def __init__(self, inputs=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.inputs = inputs

    def run(self):
        self.outputs = self.inputs

In [6]:
HelloWorld(inputs=3).write_graph(no_exec=False)
HelloWorld(name="Test01", inputs=17).write_graph(no_exec=False)
HelloWorld(name="Test02", inputs=42).write_graph(no_exec=False)

2022-01-13 21:13:48,267 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 21:13:48,268 (WARNING): Converting 06_named_nodes.ipynb to file HelloWorld.py
2022-01-13 21:13:50,693 (WARNING): --- Writing new DVC file! ---
2022-01-13 21:13:50,694 (WARNING): You will not be able to see the stdout/stderr of the process in real time!
2022-01-13 21:13:52,931 (INFO): Running stage 'HelloWorld':
> python -c "from src.HelloWorld import HelloWorld; HelloWorld.load(name='HelloWorld').run_and_save()" 
Creating 'dvc.yaml'
Adding stage 'HelloWorld' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml 'nodes\HelloWorld\.gitignore'

2022-01-13 21:13:52,932 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.

In [7]:
!dvc dag

+------------+ 
| HelloWorld | 
+------------+ 
+--------+ 
| Test01 | 
+--------+ 
+--------+ 
| Test02 | 
+--------+ 


We can now also build a Node that depends on multiple of the same Nodes

In [8]:
class FindMaximum(Node):
    deps = dvc.deps(
        [
            HelloWorld.load(),
            HelloWorld.load(name="Test01"),
            HelloWorld.load(name="Test02"),
        ]
    )
    maximum = zn.outs()

    def run(self):
        self.maximum = 0
        for node in self.deps:
            if node.outputs > self.maximum:
                self.maximum = node.outputs
                print(f"New maximum found {node.outputs}.")

In [9]:
FindMaximum().write_graph(no_exec=False)

2022-01-13 21:14:03,731 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 21:14:03,732 (WARNING): Converting 06_named_nodes.ipynb to file FindMaximum.py
2022-01-13 21:14:06,266 (WARNING): --- Writing new DVC file! ---
2022-01-13 21:14:06,267 (WARNING): You will not be able to see the stdout/stderr of the process in real time!
2022-01-13 21:14:08,825 (INFO): Running stage 'FindMaximum':
> python -c "from src.FindMaximum import FindMaximum; FindMaximum.load(name='FindMaximum').run_and_save()" 
New maximum found 3.
New maximum found 17.
New maximum found 42.
Adding stage 'FindMaximum' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add 'nodes\FindMaximum\.gitignore' dvc.lock dvc.yaml



In [10]:
!dvc dag

+------------+ 
| HelloWorld | 
+------------+ 
+--------+ 
| Test01 | 
+--------+ 
+--------+ 
| Test02 | 
+--------+ 
+-------------+  
| FindMaximum |  
+-------------+  


Using this combined Node we can e.g. find the maximum of the generated values.

In [11]:
FindMaximum.load().maximum

42

In [12]:
# Running it manually to highlight the print statements
FindMaximum.load().run()

New maximum found 3.
New maximum found 17.
New maximum found 42.


In [ ]:
temp_dir.cleanup()